In [1]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from datasets import load_dataset
import pandas as pd

In [4]:
ds = load_dataset("ArielUW/jobtitles")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


jobtitles/train.csv:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

jobtitles/validation.csv:   0%|          | 0.00/127k [00:00<?, ?B/s]

jobtitles/test.csv:   0%|          | 0.00/117k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4831 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/550 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [5]:
ds["test"]

Dataset({
    features: ['type', 'source_sentence', 'target_sentence'],
    num_rows: 500
})

In [6]:
df = ds['test'].to_pandas()

In [7]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="mongrz/neutral_job_title_rephraser_pl")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Device set to use cuda:0


In [12]:
df['generated_text'] = df['source_sentence'].apply(lambda x: pipe(x)[0]['generated_text'])

In [10]:
pipe(df['source_sentence'][0])[0]['generated_text']

'W przypadku tych ostatnich gatunków kopulacja odbywa się bez penetracji samic poprzez styk nabrzmiałych warg kloak.'

In [14]:
df.columns

Index(['type', 'source_sentence', 'target_sentence', 'generated_text'], dtype='object')

In [15]:
df['score'] = df.apply(lambda row: 1 if row['target_sentence'] == row['generated_text'] else 0, axis = 1)

In [16]:
df.to_csv('/content/drive/MyDrive/ML_class/generated_job_titles.csv', index=False)